<a href="https://colab.research.google.com/github/esmeee51/B134030044/blob/main/HW2_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 問題定義

**目標**

本專案旨在透過花萼與花瓣的四個實數特徵（長度與寬度），建立模型以預測鳶尾花的品種，包含三類：Setosa、Versicolor、Virginica

**任務類型**

多類別監督式分類（Multiclass Supervised Classification）。

**使用模型**

K-Nearest Neighbors（KNN）演算法。

**評估指標**
- Accuracy（準確率）為主要指標
- 輔以 Precision、Recall、F1-score
- 混淆矩陣（Confusion Matrix）觀察細部分類表現

## 資料收集

| 來源                   | 特徵數 | 樣本數 | 類別 |
| -------------------- | --- | --- | -- |
| scikit‑learn 內建 Iris | 4   | 150 | 3  |

特徵欄位：

- sepal length (cm)
- sepal width (cm)
- petal length (cm)
- petal width (cm)

In [ ]:
from sklearn.datasets import load_iris
iris = load_iris()
X, y = iris.data, iris.target
feature_names, target_names = iris.feature_names, iris.target_names

## 資料前處理

### 資料清理

載入資料後檢查缺失值，結果顯示無缺漏。

In [ ]:
import pandas as pd
df = pd.DataFrame(X, columns=feature_names).assign(target=y)
assert not df.isna().any().any()

Iris 無缺失

### 探索性分析

透過 seaborn.pairplot 視覺化變數分布與類別區分性。

In [ ]:
import seaborn as sns, matplotlib.pyplot as plt
sns.pairplot(
    df.replace({'target': dict(enumerate(target_names))}),
    hue="target",
    diag_kind="hist"
)
plt.show()

Setosa 與其他兩類在花瓣長、寬上分離度極高；Versicolor 與 Virginica 重疊較多。

### 資料分割

訓練測試比為 8:2，並使用 stratify 保持類別分佈一致。

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

### 特徵縮放

由於 KNN 仰賴距離計算，特徵必須標準化。


In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X)
X_scaled = scaler.transform(X)

## 模型訓練

採用 KNN 演算法訓練模型，使用 KNN，k=5，距離為歐氏距離。

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline

knn_clf = make_pipeline(
    StandardScaler(),          # 再保險放入管線，確保推論時一致
    KNeighborsClassifier(
        n_neighbors=5,         # 預設 k=5
        weights="uniform",     # 等權重
        metric="euclidean"
    )
)

knn_clf.fit(X_train, y_train)

## 模型評估

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

y_pred = knn_clf.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {acc:.3f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=target_names))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Accuracy 介於 **0.93–1.00**（視 k、距離權重而異）；Iris 是「小且規律」資料集，KNN 通常效果極佳。

## 模型調整

### 調參目標

透過交叉驗證優化以下參數：
- n_neighbors: k 值，取值範圍為 3–15。
- weights: "uniform" 或 "distance"。
- metric: 距離度量使用 "euclidean" 或 "manhattan"。

In [ ]:
from sklearn.model_selection import GridSearchCV

pipe = make_pipeline(StandardScaler(), KNeighborsClassifier())
param_grid = {
    "kneighborsclassifier__n_neighbors": range(3, 16),
    "kneighborsclassifier__weights": ["uniform", "distance"],
    "kneighborsclassifier__metric": ["euclidean", "manhattan"]
}

grid = GridSearchCV(pipe, param_grid, cv=10, scoring="accuracy", n_jobs=-1)
grid.fit(X_train, y_train)

print("最佳參數：", grid.best_params_)
print(f"CV 平均 Accuracy：{grid.best_score_:.3f}")

best_model = grid.best_estimator_

**調參要點**

| 參數            | 說明                                               |
| ------------- | ------------------------------------------------ |
| `n_neighbors` | k 值過小易受雜訊影響，過大則平滑過度；通常取 √N 或交叉驗證搜尋。              |
| `weights`     | `"uniform"`：等權；`"distance"`：距離反比權重，對類別邊界效果較好。    |
| `metric`      | Iris 常見 `euclidean`，但在特徵關係稀疏或離散時可嘗試 `manhattan`。 |

### 測試集驗證最佳模型

In [ ]:
y_pred = best_model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"\nTest Accuracy (Best Model): {acc:.3f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=target_names))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

## 模型部署

### 儲存模型

In [ ]:
import joblib
joblib.dump({
    "pipeline": best_model,      # StandardScaler + KNN
    "target_names": target_names
}, "iris_knn_pipeline.joblib")

### 推論預測

In [ ]:
import numpy as np, joblib
artifacts = joblib.load("iris_knn_pipeline.joblib")
pipe, names = artifacts["pipeline"], artifacts["target_names"]

sample = [[6.3, 2.7, 4.9, 1.8]]
pred_idx = pipe.predict(sample)[0]
print("預測品種：", names[pred_idx])

## 結論

本專案成功利用 KNN 演算法進行鳶尾花分類，透過標準化與 GridSearch 調參，在測試集上達到 95% 以上準確率。由於 Iris 資料集小且分布規律，KNN 可有效發揮距離判斷優勢。

**優點**
- 不須訓練時間，部署快速
- 實作簡單、解釋直觀
- 效果穩定，尤其適用於結構良好的小型資料集

**建議與延伸**
- 若遇大型資料集或高維特徵空間，KNN 效能可能下降，建議搭配 PCA 或其他降維技術。
- 可進一步嘗試 SVM、Random Forest 比較效能表現。